# 使用ORM操作数据

[原文](https://docs.sqlalchemy.org/en/20/tutorial/data.html)

在使用事务和 DBAPI 中，我们了解了如何与 Python DBAPI 及其事务状态进行交互的基础知识。 然后，在使用数据库元数据中，我们学习了如何使用元数据和相关对象在 SQLAlchemy 中表示数据库表、列和约束。 在本节中，我们将结合上述两个概念在关系数据库中创建、选择和操作数据。 我们与数据库的交互总是在事务方面，即使我们已经将数据库驱动程序设置为在后台使用自动提交。

- INSERT： 为了将一些数据输入数据库，我们介绍并演示了核心插入构造。从ORM角度来看，在下一部分使用ORM的数据操作中描述了插入。

- SELECT：描述Select构造，这是SQLalchemy中最常用的对象。选择核心和以ORM为中心的应用程序的SELECT构造发出的选择语句，这两个用例将在此处描述。在后面的部分中还使用查询中的关系以及ORM查询指南中的其他ORM用例。

- UPDATE && DELETE：从 Core 的角度描述 Update 和 Delete 构造的使用。ORM 特定的 UPDATE 和 DELETE 在数据操作中与 ORM 部分类似地进行了描述。

In [2]:
# 数据准备
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy import ForeignKey
from sqlalchemy import Table, Column, Integer, String

from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):  # 创建一个基类， 其他表的声明继承这个基类实现
    pass

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    test: Mapped[str]
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

class SomeTable(Base):
    __tablename__ = "some_table"
    
    x: Mapped[int] = mapped_column(nullable=True, primary_key=True)
    y: Mapped[int] = mapped_column(nullable=True)

user_table = User
address_table = Address
some_table = SomeTable


engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)
Base.metadata.create_all(engine)

2023-06-01 11:58:52,888 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-01 11:58:52,889 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-06-01 11:58:52,890 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-01 11:58:52,892 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2023-06-01 11:58:52,893 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-01 11:58:52,895 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2023-06-01 11:58:52,895 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-01 11:58:52,898 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2023-06-01 11:58:52,900 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-01 11:58:52,901 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("some_table")
2023-06-01 11:58:52,902 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-01 11:58:52,903 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("some_table")
2023-06-01 11:58:52,904 INFO sqlalchemy.engine.Eng

## INSERT 使用

使用 insert 方法构造SQL INSERT语句

插入的一个简单示例同时说明了目标表和 VALUES 子句:

大多数 SQL 表达式都可以在适当的位置进行字符串化，作为查看所产生内容的一般形式的一种手段:

In [3]:
from sqlalchemy import insert
stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")
print(stmt)
print(stmt.compile())

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)
INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


我们的 Insert 结构是一个“参数化”结构的例子，前面在发送参数中进行了说明; 要查看名称和全名绑定参数，这些也可以从已编译的结构中获得:

In [4]:
stmt.compile().params

{'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}

### 执行声明语句

调用语句时，我们可以将一行插入 user_table。在 SQL 日志中可以看到 INSERT SQL 以及绑定的参数:

In [5]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2023-06-01 11:58:52,974 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-01 11:58:52,977 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2023-06-01 11:58:52,979 INFO sqlalchemy.engine.Engine [generated in 0.00419s] ('spongebob', 'Spongebob Squarepants')
2023-06-01 11:58:52,980 INFO sqlalchemy.engine.Engine COMMIT


在上面的简单形式中，INSERT 语句不返回任何行，如果只插入一行，它通常包含返回有关该行在 INSERT 期间生成的列级默认值的信息的能力，最常见的是一个整数主键值。在上述情况下，SQLite 数据库中的第一行通常会为第一个整数主键值返回1，我们可以使用 CursorResult.insert _ 置入主键访问器获取该值:

In [6]:
result.inserted_primary_key

(1,)

可能由多个主键，所以这是一个元组。

### INSERT 通常自动生成“ value”子句

上面的示例使用 INSERT.VALUES ()方法显式创建 SQL INSERT 语句的 VALUES 子句。如果我们实际上没有使用 INSERT.values () ，而只是打印出一个“空”语句，那么对于表中的每一列，我们都会得到一个 INSERT:

如果我们采用一个没有`insert.values（）`调用并执行它而不是打印的插入构造，则该语句将根据我们传递给`Connection.execute.execute（）`方法，并且仅包括与通过参数相关的列。实际上，这是插入用来插入行而无需输入显式值子句的通常方式。下面的示例说明了一个两列插入语句正在执行使用参数列表执行：


In [7]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"},
        ],
    )
    conn.commit()

2023-06-01 11:58:53,023 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-01 11:58:53,024 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2023-06-01 11:58:53,025 INFO sqlalchemy.engine.Engine [generated in 0.00268s] [('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star')]
2023-06-01 11:58:53,026 INFO sqlalchemy.engine.Engine COMMIT


上面的执行具有“executemany”形式，首先在发送多个参数中进行了说明，但是与使用 text() 构造时不同的是，我们不必拼出任何 SQL。 通过将字典或字典列表与 Insert 构造一起传递给 Connection.execute() 方法，Connection 确保传递的列名将自动在 Insert 构造的 VALUES 子句中表达。

### 高级用法

以下是一个更高级的示例，说明了如何明确使用`insert.values（）`方法，同时包括从参数生成的其他值。构建了标量子查询，利用下一节中引入的`select（）`构造，并使用使用`BindParam（）`构造建立的显式绑定参数名称进行了子查询中使用的参数。



### INSERT…RETURNING

支持后端的返回子句自动使用，以检索最后插入的主要键值以及服务器默认值的值。但是，也可以使用insert.turn.turning（）方法明确指定返回子句；在这种情况下，执行语句后返回的结果对象具有可以获取的行：

In [8]:
insert_stmt = insert(address_table).returning(
    address_table.id, address_table.email_address
)
print(insert_stmt)

INSERT INTO address (id, email_address, user_id, test) VALUES (:id, :email_address, :user_id, :test) RETURNING address.id, address.email_address


它还可以与 INSERT.FROM _ SELECT ()组合，如下面的示例所示，该示例构建在 INSERT... FROM SELECT:

In [9]:
from sqlalchemy import select
select_stmt = select(user_table.id, user_table.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt.returning(address_table.id, address_table.email_address))

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account RETURNING address.id, address.email_address


> UPDATE 和 DELETE 语句也支持 RETURING 特性，这些语句将在本教程后面介绍。
> 对于 INSERT 语句，可以对单行语句和同时对多行进行 INSERT 的语句使用 RETURING 特性。
> 支持带有 RETURING 的多行 INSERT 是方言特有的，但是支持 SQLAlchemy 中包含的所有支持 RETURING 的方言。
> 有关此特性的背景信息，请参阅 INSERT 语句的“插入多个值”行为一节。

### INSERT…FROM SELECT

INSERT 的一个较少使用的特性，但是为了完整起见，INSERT 构造可以组成一个 INSERT，它使用 INSERT.from _ SELECT ()方法直接从 SELECT 获取行。
此方法接受 select ()构造，下一节将讨论该构造，以及实际 INSERT 中要针对的列名列表。在下面的例子中，行被添加到地址表中，这些行是从 user _ account 表中的行派生出来的，为每个用户提供了 aol.com 上的免费电子邮件地址:



In [10]:
select_stmt = select(user_table.id, user_table.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt)

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account


当需要将数据从数据库的某个其他部分直接复制到一组新的行中时，不需要从客户机实际获取和重新发送数据，就可以使用此构造。

## 使用 SELECT 语句

对于 Core 和 ORM，select()函数生成一个 SELECT 构造，用于所有 SELECT 查询。
传递给 Core 中的 Connection.execute()和 ORM 中的 Session.execute()等方法时，会在当前事务中发出一条 SELECT 语句，并通过返回的 Result 对象提供结果行。



### The select() SQL Expression Construct¶

select()构造使用与 insert()构造相同的方式构建语句，使用一种生成方法，其中每个方法在对象上构建更多的状态。
与其他 SQL 构造一样，它可以在适当的位置进行字符串化:

In [11]:
from sqlalchemy import select
stmt = select(user_table).where(user_table.name == "spongebob")
print(stmt)


SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


与所有其他语句级 SQL 构造一样，为了实际运行语句，我们将其传递给一个执行方法。
因为 SELECT 语句返回行，所以我们总是可以迭代结果对象来返回 Row 对象:


In [12]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2023-06-01 11:58:53,153 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-01 11:58:53,155 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2023-06-01 11:58:53,156 INFO sqlalchemy.engine.Engine [generated in 0.00338s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
2023-06-01 11:58:53,157 INFO sqlalchemy.engine.Engine ROLLBACK


当使用 ORM 时，特别是使用一个针对 ORM 实体组成的 select ()结构时，我们将希望使用 Session 上的 Session.execute ()方法来执行它;
使用这种方法，我们继续从结果中获取 Row 对象，但是这些行现在能够包含完整的实体，比如 User 类的实例，作为每一行中的单独元素:

简单来说，使用Session 查询的结果将会直接被影射为Python对象。

In [13]:
from sqlalchemy.orm.session import Session

stmt = select(User).where(User.name == "spongebob")
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2023-06-01 11:58:53,175 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-01 11:58:53,178 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2023-06-01 11:58:53,179 INFO sqlalchemy.engine.Engine [generated in 0.00131s] ('spongebob',)
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)
2023-06-01 11:58:53,182 INFO sqlalchemy.engine.Engine ROLLBACK


> **从表与ORM类中select（）**  
>
> 虽然在这些例子中生成的 SQL 看起来是相同的，无论我们调用 `select(user_table)`还是 `select(User)` ，在更一般的情况下，它们不一定呈现相同的东西，因为一个 ORM 映射的类可能被映射到除了表之外的其他类型的"selectables"。针对 ORM 实体的 select()还指示 ORM 映射的实例应该在结果中返回，而从 Table 对象进行 SELECTing 时则不是这种情况。



### Setting the COLUMNS and FROM clause

`select()`函数接受表示任意数量的 Column 和/或 Table 表达式的位置元素，以及范围广泛的兼容对象，这些对象被解析为一个 SQL 表达式列表，从中进行 SELECTed，并作为结果集中的列返回。这些元素还可以在更简单的情况下创建 FROM 子句，这是从传递的列和类似表的表达式中推断出来的:

In [14]:
print(select(user_table))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


要使用 Core 方法从单个列中选择 SELECT，可以从 Table.c 访问器访问 Column 对象，并且可以直接发送; FROM 子句将被推断为由这些列表示的所有 Table 和其他 FromClar 对象的集合:



In [15]:
print(select(user_table.name, user_table.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


或者，当使用任何 FromClause.c 集合(如 Table)时，可以使用字符串名称元组为 select ()指定多个列:

注意：这只适用于 Table()构造的表对象。

In [16]:
print(select(user_table["name", "fullname"]))

TypeError: <class '__main__.User'> is not a generic class

### Selecting ORM Entities and Columns¶

ORM 实体，比如 User 类，以及它上面的列映射属性，比如 User.name，也参与到表示表和列的 SQL Expression Language 系统中。
下面展示了一个来自 User 实体的 SELECTing 示例，它最终呈现的方式与直接使用 User_table 的方式相同:

In [17]:
print(select(User))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


当使用 ORM Session.execute ()方法执行上述语句时，与 User _ table 相比，从 User 这样的完整实体中进行选择有一个重要的区别，那就是实体本身在每行中作为单个元素返回。也就是说，当我们从上面的语句中获取行时，因为要获取的列表中只有 User 实体，所以我们返回只有一个元素的 Row 对象，它包含 User 类的实例:



In [18]:
row = session.execute(select(User)).first()
row

2023-06-01 11:59:08,508 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-01 11:59:08,510 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2023-06-01 11:59:08,513 INFO sqlalchemy.engine.Engine [generated in 0.00355s] ()


(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)

上面的 Row 只有一个元素，表示 User 实体:



In [19]:
row[0]

User(id=1, name='spongebob', fullname='Spongebob Squarepants')

一个强烈推荐的方便的方法是使用 Session.scalars()方法直接执行语句，这个方法将返回一个 ScalarResult 对象，它立即传递每一行的第一个“列”，在这个例子中，User 类的实例:

In [20]:
user = session.scalars(select(User)).first()
user

2023-06-01 11:59:08,559 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2023-06-01 11:59:08,560 INFO sqlalchemy.engine.Engine [cached since 0.05075s ago] ()


User(id=1, name='spongebob', fullname='Spongebob Squarepants')

或者，我们可以使用类绑定属性在结果行中选择 ORM 实体的单个列作为不同的元素;
当这些属性被传递给诸如 select()之类的构造时，它们被解析为 Column 或由每个属性表示的其他 SQL 表达式:



In [21]:
print(select(User.name, User.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


当我们使用 Session.execute()调用这个语句时，我们现在接收的行中每个值都有单独的元素，每个元素对应一个单独的列或其他 SQL 表达式:



In [22]:
row = session.execute(select(User.name, User.fullname)).first()
row

2023-06-01 11:59:08,608 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2023-06-01 11:59:08,609 INFO sqlalchemy.engine.Engine [generated in 0.00105s] ()


('spongebob', 'Spongebob Squarepants')

这些方法也可以混合使用，如下所示，我们将 User 实体的 name 属性作为行的第一个元素进行 SELECT，并将其与第二个元素中的完整 Address 实体组合在一起:

In [23]:
session.execute(
    select(User.name, Address).where(User.id == Address.user_id).order_by(Address.id)
).all()

2023-06-01 11:59:08,634 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.id, address.email_address, address.user_id, address.test 
FROM user_account, address 
WHERE user_account.id = address.user_id ORDER BY address.id
2023-06-01 11:59:08,637 INFO sqlalchemy.engine.Engine [generated in 0.00237s] ()


[]

### 从标记的SQL表达式选择¶

Label()方法以及 ORM 属性上可用的同名方法提供了一个列或表达式的 SQL 标签，允许它在结果集中具有特定的名称。当按名称引用结果行中的任意 SQL 表达式时，这会很有帮助:



In [24]:
from sqlalchemy import func, cast
stmt = select(
    ("Username: " + user_table.name).label("username"),
).order_by(user_table.name)
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.username}")

2023-06-01 11:59:08,660 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-01 11:59:08,663 INFO sqlalchemy.engine.Engine SELECT ? || user_account.name AS username 
FROM user_account ORDER BY user_account.name
2023-06-01 11:59:08,666 INFO sqlalchemy.engine.Engine [generated in 0.00613s] ('Username: ',)
Username: patrick
Username: sandy
Username: spongebob
2023-06-01 11:59:08,668 INFO sqlalchemy.engine.Engine ROLLBACK


### Selecting with Textual Column Expressions¶

当我们使用 select()函数构造 Select 对象时，我们通常会向其传递一系列使用表元数据定义的 Table 和 Column 对象，或者当使用 ORM 时，我们可能会发送表示表列的 ORM 映射属性。
但是，有时也需要在语句内部生成任意的 SQL 块，例如常量字符串表达式，或者只生成一些任意的 SQL，这样可以更快地按字面意思写入。

在 Working with Transactions 和 DBAPI 中引入的 text()构造实际上可以直接嵌入 SELECT 构造中，比如下面我们生成一个硬编码的字符串文字“ some words”并将其嵌入到 SELECT 语句中:


In [25]:
from sqlalchemy import text
stmt = select(text("'some phrase'"), user_table.name).order_by(user_table.name)
with engine.connect() as conn:
    print(conn.execute(stmt).all())

2023-06-01 11:59:08,688 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-01 11:59:08,689 INFO sqlalchemy.engine.Engine SELECT 'some phrase', user_account.name 
FROM user_account ORDER BY user_account.name
2023-06-01 11:59:08,692 INFO sqlalchemy.engine.Engine [generated in 0.00434s] ()
[('some phrase', 'patrick'), ('some phrase', 'sandy'), ('some phrase', 'spongebob')]
2023-06-01 11:59:08,693 INFO sqlalchemy.engine.Engine ROLLBACK


尽管 text() 结构在大多数地方都可以用来注入文字 SQL 短语，但我们实际上多半是在处理每个表示单个列表达式的文本单元。
在这种常见的情况下，我们可以使用 literal_column() 结构从文本片段中获得更多的功能。这个对象类似于 text () ，只不过它不是表示任意形式的 SQL，而是显式地表示一个“列”，然后可以在子查询和其他表达式中标记和引用它:


In [26]:
from sqlalchemy import literal_column
stmt = select(literal_column("'some phrase'").label("p"), user_table.name).order_by(
    user_table.name
)
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.p}, {row.name}")

2023-06-01 11:59:08,718 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-01 11:59:08,720 INFO sqlalchemy.engine.Engine SELECT 'some phrase' AS p, user_account.name 
FROM user_account ORDER BY user_account.name
2023-06-01 11:59:08,721 INFO sqlalchemy.engine.Engine [generated in 0.00264s] ()
some phrase, patrick
some phrase, sandy
some phrase, spongebob
2023-06-01 11:59:08,723 INFO sqlalchemy.engine.Engine ROLLBACK


请注意，在这两种情况下，使用text（）或trielal_column（）时，我们正在编写句法SQL表达式，而不是文字值。
因此，我们必须包括我们想要看到的SQL所需的任何引用或语法。

### The WHERE clause¶

SQLalchemy允许我们通过与列和类似对象结合使用标准Python操作员来组成SQL表达式，例如名称='squidward'或user_id> 10。
对于布尔表达式，大多数python运算符，例如==，!=，<，>=等。
生成新的sql表达式对象，而不是普通的布尔值true/false值：

In [27]:
print(user_table.name == "squidward")

print(address_table.user_id > 10)

user_account.name = :name_1
address.user_id > :user_id_1


我们可以使用类似的表达式通过将结果对象传递给select.where（）方法来生成Where子句：

In [28]:
print(select(user_table).where(user_table.name == "squidward"))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


为了产生由 AND 连接的多个表达式，Select.where ()方法可以被调用任意次数:

In [29]:
print(
    select(address_table.email_address)
    .where(user_table.name == "squidward")
    .where(address_table.user_id == user_table.id)
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


对 Select.where ()的单个调用也接受具有相同效果的多个表达式:

In [30]:
print(
    select(address_table.email_address).where(
        user_table.name == "squidward",
        address_table.user_id == user_table.id,
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


“AND”和“OR”连词都可以直接使用 and_() or_()函数，下面用 ORM 实体说明:

In [31]:
from sqlalchemy import and_, or_
print(
    select(Address.email_address).where(
        and_(
            or_(User.name == "squidward", User.name == "sandy"),
            Address.user_id == User.id,
        )
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE (user_account.name = :name_1 OR user_account.name = :name_2) AND address.user_id = user_account.id


对于单个实体的简单“相等”比较，还有一个流行的方法 Select.filter _ by () ，它接受与列键或 ORM 属性名匹配的关键字参数。它将根据最左边的 FROM 子句或最后加入的实体进行筛选:

In [32]:
print(select(User).filter_by(name="spongebob", fullname="Spongebob Squarepants"))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1 AND user_account.fullname = :fullname_1


### Explicit FROM clauses and JOINs

如前所述，FROM 子句通常是根据我们在 column 子句中设置的表达式以及 Select 的其他元素推断出来的。

如果我们从“Column”子句中的特定表中设置一个列，则将该表也放在"FROM"子句中：


In [33]:
print(select(user_table.name))

SELECT user_account.name 
FROM user_account


如果我们将两个表中的列放在一起，那么我们将得到一个逗号分隔的 FROM 子句:


In [34]:
print(select(user_table.name, address_table.email_address))

SELECT user_account.name, address.email_address 
FROM user_account, address


为了将这两个表连接在一起，我们通常在SELECT上使用两种方法之一。
第一个是select.join_from（）方法，它允许我们明确指示加入的左侧和右侧：

In [35]:
print(
    select(user_table.name, address_table.email_address).join_from(
        user_table, address_table
    )
)

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


另一个是select.join（）方法，它仅表示联接的右侧，左手侧是：


In [37]:
print(select(user_table.name, address_table.email_address).join(address_table))

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id
